In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [0]:
import sys
sys.path.append ('../')
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [0]:
import seaborn as sns

In [0]:
PATH = "Titanic/"
trainDf = pd.read_csv(f'{PATH}train.csv', low_memory=False)
testDf = pd.read_csv(f'{PATH}test.csv', low_memory=False)

In [0]:
testID = testDf.PassengerId

In [0]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [0]:
df_main = pd.concat([trainDf, testDf], axis=0, ignore_index= True)

C:\Users\M.Hamza.Naveed\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
i = df_main[df_main.Age.isnull()].index
i

Int64Index([   5,   17,   19,   26,   28,   29,   31,   32,   36,   42,
            ...
            1256, 1257, 1271, 1273, 1275, 1299, 1301, 1304, 1307, 1308],
           dtype='int64', length=263)

In [0]:
missing = (df_main.isnull().sum()[df_main.isnull().sum()>0]).sort_values(ascending=False)
plt.figure(figsize=(15,10))
sns.barplot(x=missing[missing>0].index, y=missing[missing>0].values)
plt.xticks(rotation=90)
plt.grid(b=True, color='lightgrey')

In [0]:
display_all(df_main.isnull().sum().sort_index(ascending = False))

isChild            0
isAged             0
isAdult            0
Ticket             0
Survived         418
SibSp              0
Sex                0
Pclass             0
PassengerId        0
Parch              0
Name               0
FarePerPerson      1
Fare               1
FamilyTotal        0
Embarked           2
Cabin            998
Age                0
dtype: int64

In [0]:
df_main['FamilyTotal'] = df_main.Parch + df_main.SibSp + 1
df_main['FarePerPerson'] = df_main.Fare/df_main.FamilyTotal

In [0]:
c=[]
for idx in i:
    if ('Mr.'in df_main.iloc[idx].Name) | ('Mrs.'in df_main.iloc[idx].Name):
        c.append(idx)
len(c)


203

In [0]:
for idx in c:
    df_main.at[idx, 'Age'] = 30
        

In [0]:
df_main.Age.isnull().sum()

0

In [0]:
df_main.Age.fillna(15, inplace=True)

In [0]:
df_main['isAdult'] = df_main.Age.map(lambda x: 0 if ((x<16) | (x>50)) else 1)
df_main['isAged'] = df_main.Age.map(lambda x: 0 if x<50 else 1)

In [0]:
df_main['isChild'] = df_main.Age.map(lambda x: 0 if x>15 else 1)

In [0]:
train_cats(df_main)

In [0]:
df_main.drop('PassengerId', axis = 1, inplace = True)

In [0]:
df, y, nas = proc_df(df_main, 'Survived', max_n_cat=5)

In [0]:
dfTest = df[891:]
df = df[:891]
y = y[:891]
df.shape, y.shape, df_main.shape

((891, 22), (891,), (1309, 16))

In [0]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 100 
n_trn = len(df)-n_valid #1459

X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((791, 22), (791,), (100, 22), (100,))

In [0]:
m = RandomForestClassifier(n_estimators = 140, min_samples_leaf=1,  criterion = 'entropy', max_features='log2', n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.0, 0.37416573867739417, 1.0, 0.86, 0.8394437420986094]


In [0]:
fi = rf_feat_importance(m, df); fi[:10]

In [0]:
def plot_fi(fi): return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)
plot_fi(fi[:])

In [0]:
def get_oob(df):
    m = RandomForestRegressor(n_estimators=30, min_samples_leaf=5, max_features=0.6, n_jobs=-1, oob_score=True)
    x, _ = split_vals(df, n_trn)
    m.fit(x, y_train)
    return m.oob_score_

In [0]:
get_oob(df)

In [0]:
# Predicting the validation set results
y_pred = m.predict(X_valid)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_valid, y_pred)

In [0]:
cm

array([[59,  5],
       [17, 19]], dtype=int64)

In [0]:
from sklearn.model_selection import cross_val_score

In [0]:
accuracies = cross_val_score(estimator = xgb1, X=X_train, y=y_train, cv=10, n_jobs=-1)
accuracies.mean()

0.8231280428432328

In [0]:
accuracies

array([0.825     , 0.825     , 0.7625    , 0.7875    , 0.875     ,
       0.81012658, 0.79487179, 0.84615385, 0.87179487, 0.83333333])

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
parameters = [{'n_estimators': [100,120,140], 
               'min_samples_leaf': [1,3,5],
               'max_features': [0.5, 'auto', 'sqrt', 'log2'],
               'criterion': ['entropy']},
              ]

In [0]:
gridSearch = GridSearchCV(estimator=m, param_grid=parameters, scoring='accuracy', cv=10, n_jobs=-1)
gridSearch = gridSearch.fit(X_train,y_train)

bestAcc = gridSearch.best_score_
bestParams = gridSearch.best_params_

In [0]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
xgb1.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)

In [0]:
print_score(xgb1)

[0.0, 0.4, 1.0, 0.84]


In [0]:
from xgboost import XGBClassifier

In [0]:
m = XGBClassifier( colsample_bytree = 0.7,
  learning_rate = 0.05,
  max_depth = 6,
  min_child_weight = 11,
  n_estimators = 5,
  nthread = 4,
  seed = 1337,
  silent = 1,
  subsample = 0.8)

m.fit(X_train,y_train)
print_score(m)

[0.4340151529277784, 0.469041575982343, 0.8116308470290771, 0.78]


In [0]:
sub = m.predict(dfTest).astype(int)
submission = pd.DataFrame()
submission['PassengerId'] = testID
submission['Survived'] = sub
submission.head()
submission.to_csv("Titanic11.csv", index=False)

In [0]:
display_all(df_main.T)

In [0]:
# count = 0
# for j in range(0,len(df_main)):
#     if type(df_main.Cabin[j]) != str:
#         count +=1
# count

#         n = df_main.Name[j]
#         n = n.split(',')
#         n = n[0]

In [0]:
for j in range(0,len(tempDf)):
    if type(tempDf.Cabin[j]) == float:
        t = tempDf.Ticket[j]
        for jj in range(0,len(tempDf)):
            if j == jj:
                continue
            if (tempDf.Ticket[jj] == t) & (type(tempDf.Cabin[jj]) == str) :
                tempDf.Cabin[j] = tempDf.Cabin[jj]

In [0]:
df_main[(df_main.Ticket == 'W./C. 6608')]

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,FamilyTotal,FarePerPerson,isAdult,isAged,isChild
86,16.0,NaN,S,34.375,"Ford, Mr. William Neal",3,87,3,male,1,0.0,W./C. 6608,5,6.875,1,0,1
147,9.0,NaN,S,34.375,"Ford, Miss. Robina Maggie ""Ruby""",2,148,3,female,2,0.0,W./C. 6608,5,6.875,0,0,1
436,21.0,NaN,S,34.375,"Ford, Miss. Doolina Margaret ""Daisy""",2,437,3,female,2,0.0,W./C. 6608,5,6.875,1,0,0
736,48.0,NaN,S,34.375,"Ford, Mrs. Edward (Margaret Ann Watson)",3,737,3,female,1,0.0,W./C. 6608,5,6.875,1,0,0
1058,18.0,NaN,S,34.375,"Ford, Mr. Edward Watson",2,1059,3,male,2,NaN,W./C. 6608,5,6.875,1,0,0


In [0]:
display_all(df_main[(df_main.Pclass == 1) & (df_main.Cabins.str.contains('A'))])